
<div  style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://raw.githubusercontent.com/derar-alhussein/Databricks-Certified-Data-Engineer-Associate/main/Includes/images/bookstore_schema.png" alt="Databricks Learning" style="width: 600">
</div>

In [0]:
%run ../Includes/Copy-Datasets


## Parsing JSON Data

In [0]:
SELECT * FROM customers

In [0]:
DESCRIBE customers

In [0]:
SELECT customer_id, profile:first_name, profile:address:country 
FROM customers

In [0]:
SELECT from_json(profile) AS profile_struct
  FROM customers;

In [0]:
SELECT profile 
FROM customers 
LIMIT 1

In [0]:
CREATE OR REPLACE TEMP VIEW parsed_customers AS
  SELECT customer_id, from_json(profile, schema_of_json('{"first_name":"Thomas","last_name":"Lane","gender":"Male","address":{"street":"06 Boulevard Victor Hugo","city":"Paris","country":"France"}}')) AS profile_struct
  FROM customers;
  
SELECT * FROM parsed_customers

In [0]:
DESCRIBE parsed_customers

In [0]:
SELECT customer_id, profile_struct.first_name, profile_struct.address.country
FROM parsed_customers

In [0]:
CREATE OR REPLACE TEMP VIEW customers_final AS
  SELECT customer_id, profile_struct.*
  FROM parsed_customers;
  
SELECT * FROM customers_final

In [0]:
SELECT order_id, customer_id, books
FROM orders

## Explode Function

In [0]:
SELECT order_id, customer_id, explode(books) AS book 
FROM orders

## Collecting Rows

In [0]:
SELECT customer_id,
  collect_set(order_id) AS orders_set,
  collect_set(books.book_id) AS books_set
FROM orders
GROUP BY customer_id


##Flatten Arrays

In [0]:
SELECT customer_id,
  collect_set(books.book_id) As before_flatten,
  array_distinct(flatten(collect_set(books.book_id))) AS after_flatten
FROM orders
GROUP BY customer_id


##Join Operations

In [0]:
CREATE OR REPLACE VIEW orders_enriched AS
SELECT *
FROM (
  SELECT *, explode(books) AS book 
  FROM orders) o
INNER JOIN books b
ON o.book.book_id = b.book_id;

SELECT * FROM orders_enriched

## Set Operations

In [0]:
CREATE OR REPLACE TEMP VIEW orders_updates
AS SELECT * FROM parquet.`${dataset.bookstore}/orders-new`;

SELECT * FROM orders 
UNION 
SELECT * FROM orders_updates 

In [0]:
SELECT * FROM orders 
INTERSECT 
SELECT * FROM orders_updates 

In [0]:
SELECT * FROM orders 
MINUS 
SELECT * FROM orders_updates 

## Reshaping Data with Pivot

In [0]:
CREATE OR REPLACE TABLE transactions AS

SELECT * FROM (
  SELECT
    customer_id,
    book.book_id AS book_id,
    book.quantity AS quantity
  FROM orders_enriched
) PIVOT (
  sum(quantity) FOR book_id in (
    'B01', 'B02', 'B03', 'B04', 'B05', 'B06',
    'B07', 'B08', 'B09', 'B10', 'B11', 'B12'
  )
);

SELECT * FROM transactions